# Using a Modflow UZF model as a stressmodel in Pastas

Please note that the showcased workflow is not how we intend to keep things. Major breaking changes in the next release.

In [ ]:
from pathlib import Path

import flopy
import pandas as pd
import pastas as ps
from pastas.timer import SolveTimer

from pastas_plugins import modflow as ppmf

bindir = Path("bin")
if not (bindir / "mf6").exists():
    bindir.mkdir(parents=True, exist_ok=True)
    flopy.utils.get_modflow(bindir, repo="modflow6")

## Data Groundwater Article

In [ ]:
ds = ps.load_dataset("collenteur_2019")
head = ds["head"].squeeze()
prec = ds["rain"].squeeze().resample("D").asfreq().fillna(0.0)
evap = ds["evap"].squeeze()

tmin = pd.Timestamp("2003-01-01")
tmax = pd.Timestamp("2018-12-25")

ps.plots.series(head, [prec, evap], hist=False);

### Standard exponential model

In [ ]:
mlexp = ps.Model(head)
mlexp.add_stressmodel(
    ps.RechargeModel(prec=prec, evap=evap, rfunc=ps.Exponential(), name="exp")
)
mlexp.solve(
    tmin=tmin,
    tmax=tmax,
)
mlexp.plot()

## FLEX model

In [ ]:
mlflex = ps.Model(head)
rm = ps.RechargeModel(
    prec=prec.multiply(1e3),
    evap=evap.multiply(1e3),
    rfunc=ps.Exponential(),
    recharge=ps.rch.FlexModel(),
    name="flex",
)
mlflex.add_stressmodel(rm)
mlflex.set_parameter(f"{rm.name}_srmax", vary=False)
mlflex.solve(tmin=tmin, tmax=tmax)
mlflex.plot()

### UZF model calibrated

In [ ]:
mlmf = ps.Model(head, name="uzftest")
mfmodel = ppmf.ModflowModel(
    model=mlmf,
    exe_name=bindir / "mf6",
    sim_ws=Path("uzftest"),
    silent=True,
    solver_kwargs=dict(
        print_option="summary",
        outer_dvclose=3e-2,
        outer_maximum=300,
        under_relaxation="dbd",
        linear_acceleration="BICGSTAB",
        under_relaxation_theta=0.7,
        under_relaxation_kappa=0.08,
        under_relaxation_gamma=0.05,
        under_relaxation_momentum=0.0,
        inner_dvclose=3e-2,
        rcloserecord="1000.0 strict",
        inner_maximum=500,
        relaxation_factor=0.97,
        number_orthogonalizations=2,
        preconditioner_levels=8,
        preconditioner_drop_tolerance=0.001,
    ),
)

# shorten the warmup to speed up the modflow calculation somewhat.
mlmf.settings["warmup"] = pd.Timedelta(days=4 * 365)
ghb = ppmf.ModflowGhb()
uzf = ppmf.ModflowUzf(prec, evap, ntrailwaves=35, nwavesets=100)
mfmodel.add_modflow_package([ghb, uzf])

mlmf.solve(diff_step=1e-4, max_nfev=3)

### Compare

In [ ]:
ax = mlexp.observations().plot(
    marker=".", color="k", linestyle="None", legend=False, figsize=(7.5, 6)
)
mlexp.simulate().plot(ax=ax, label="Exponential")
mlflex.simulate().plot(ax=ax, label="FlexModel")
mlmf.simulate().plot(ax=ax, label="UZF")
ax.legend(ncol=4, loc="upper right")
ax.set_xlim(tmin + pd.Timedelta(days=2 * 365), tmax)

## Synthetic data

In [ ]:
ds = ps.load_dataset("vonk_2024")
head = ds["heads"].loc[:, ("SandyLoam", "D85", "x00")].rename("usg_head").iloc[1:]
prec = ds["stresses"].loc[:, "prec [m/d]"].rename("prec")
evap = ds["stresses"].loc[:, "evap [m/d]"].rename("evap")

tmin = pd.Timestamp("2000-01-01")
tmax = head.index[-1]

### UZF model with parameters we "know" from USG

In [ ]:
mlu = ps.Model(head)
mlu.del_constant()

mf = ppmf.ModflowUzf(
    exe_name=bindir / "mf6",
    sim_ws="mf_files/mfuzf_usg",
    head=head,
    ntrailwaves=35,
    nwavesets=100,
)
sm = ppmf.ModflowModel(prec, evap, modflow=mf, name="mfsm")
mlu.add_stressmodel(sm)

# only parameter we don't know
mlu.set_parameter(f"{sm.name}_c", initial=100.0)  # select by hand
# plugin "true" parameters from modflow usg model
mlu.set_parameter(f"{sm.name}_d", initial=8.5)  # level in canal
mlu.set_parameter(
    f"{sm.name}_s", initial=0.271
)  # specific yield from theta_s - theta_fc
mlu.set_parameter(f"{sm.name}_height", initial=1.5)  # freeboard
mlu.set_parameter(f"{sm.name}_vks", initial=1.0061)  # m/day
mlu.set_parameter(f"{sm.name}_thtr", initial=0.065)  # theta_r
mlu.set_parameter(f"{sm.name}_thts", initial=0.41, pmax=0.45)  # theta_s
mlu.set_parameter(f"{sm.name}_eps", initial=4.954)  # brooks in mfusg
mlu.set_parameter(f"{sm.name}_extdpfrac", initial=1.0 / 1.5)  # root depth 1 m

mlu.plot()

### UZF model calibrated

In [ ]:
with SolveTimer() as st:
    mlu.solve(callback=st.timer, diff_step=0.1)

In [ ]:
mlu.plot()